In [66]:
import pandas as pd

def load_data(path: str):
  years = list(range(2005, 2020))
  year2index = { y : i for (i, y) in enumerate(years) }
  data = [pd.read_excel(path, sheet_name=str(y)) for y in years]
  return data, years, year2index

data, years, year2index = load_data("./data/Brazilian Immigrants.xlsx")
data[year2index[2005]].head()


,Type,State Code,1.0,2.0,4.0,5.0,6.0,8.0,9.0,10.0,...,41.0,42.0,44.0,45.0,47.0,48.0,49.0,51.0,53.0,55.0
0,Population,Population,2030.0,88.0,1288.0,861.0,25111.0,1339.0,14021.0,296.0,...,571.0,7019.0,1143.0,2339.0,1355.0,7921.0,2904.0,6471.0,2505.0,804.0
1,Age,Age 0 to 4,78.0,0.0,0.0,0.0,305.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20.0,0.0,87.0,0.0,0.0
2,Age,Age 05 to 09,172.0,0.0,0.0,116.0,1039.0,84.0,333.0,0.0,...,0.0,157.0,0.0,0.0,0.0,283.0,182.0,1180.0,0.0,85.0
3,Age,Age 10 to 19,242.0,0.0,0.0,0.0,2920.0,0.0,1309.0,0.0,...,0.0,748.0,0.0,119.0,0.0,382.0,360.0,439.0,467.0,213.0
4,Age,Age 20 to 24,0.0,0.0,67.0,0.0,1910.0,0.0,2153.0,104.0,...,120.0,795.0,75.0,577.0,0.0,831.0,154.0,394.0,97.0,0.0


In [67]:
def load_state_dict(path: str):
  data = pd.read_excel(path, sheet_name="State code")
  state_dict = dict()

  for i in data.index:
    abbrev = data["State Abbreviation"][i].lower()
    name = data["State Name"][i].lower()
    code = data["FIPS Code"][i]
    
    state_dict[code] = {
      "name": name,
      "abbrev": abbrev
    }

  return state_dict

state_dict = load_state_dict("./data/Brazilian Immigrants.xlsx")
state_dict

{1.0: {'name': 'alabama', 'abbrev': 'al'},
 2.0: {'name': 'alaska', 'abbrev': 'ak'},
 4.0: {'name': 'arizona', 'abbrev': 'az'},
 5.0: {'name': 'arkansas', 'abbrev': 'ar'},
 6.0: {'name': 'california', 'abbrev': 'ca'},
 8.0: {'name': 'colorado', 'abbrev': 'co'},
 9.0: {'name': 'connecticut', 'abbrev': 'ct'},
 10.0: {'name': 'delaware', 'abbrev': 'de'},
 11.0: {'name': 'district of columbia', 'abbrev': 'dc'},
 12.0: {'name': 'florida', 'abbrev': 'fl'},
 13.0: {'name': 'georgia', 'abbrev': 'ga'},
 15.0: {'name': 'hawaii', 'abbrev': 'hi'},
 16.0: {'name': 'idaho', 'abbrev': 'id'},
 17.0: {'name': 'illinois', 'abbrev': 'il'},
 18.0: {'name': 'indiana', 'abbrev': 'in'},
 19.0: {'name': 'iowa', 'abbrev': 'ia'},
 20.0: {'name': 'kansas', 'abbrev': 'ks'},
 21.0: {'name': 'kentucky', 'abbrev': 'ky'},
 22.0: {'name': 'louisiana', 'abbrev': 'la'},
 23.0: {'name': 'maine', 'abbrev': 'me'},
 24.0: {'name': 'maryland', 'abbrev': 'md'},
 25.0: {'name': 'massachusetts', 'abbrev': 'ma'},
 26.0: {'name':

In [68]:
def query_by_attr_year(attr: str, year: int):
  df = data[year2index[year]]
  row = df.loc[df.iloc[:, 1] == attr]
  return row

row = query_by_attr_year("Population", 2006)
row.head()

,Type,State Code,1.0,4.0,5.0,6.0,8.0,9.0,10.0,11.0,...,45.0,46.0,47.0,48.0,49.0,50.0,51.0,53.0,54.0,55.0
0,Population,Population,1370.0,1435.0,139.0,31903.0,1250.0,18264.0,612.0,460.0,...,1966.0,48.0,953.0,11292.0,2867.0,108.0,7458.0,2282.0,279.0,453.0


In [72]:
def query_by_attr_year_state(attr: str, year: int, state: int):
  row = query_by_attr_year(attr, year)
  return row[state].to_numpy()[0] if state in row.columns else None

print(query_by_attr_year_state("Female", 2006, 1))
print(query_by_attr_year_state("Female", 2006, 2))

840.0
None


In [73]:
def query_attr_over_years_by_state(attr: str, state: int):
  return[query_by_attr_year_state(attr, y, state) for y in years]

query_attr_over_years_by_state("Population", 2)

[88.0,
 None,
 156.0,
 138.0,
 481.0,
 99.0,
 None,
 63.0,
 554.0,
 123.0,
 44.0,
 147.0,
 209.0,
 340.0,
 267.0]

In [153]:
from typing import List, Optional
import pyecharts.options as opts
from pyecharts.charts import Grid, Line
import os

def compare_states(attr: str, states: List[int], save_name: Optional[str] = None):
  values_by_state = [query_attr_over_years_by_state(attr, state) for state in states]

  line = (
    Line()
    .add_xaxis(xaxis_data=[str(y) for y in years])
    .set_global_opts(
      title_opts=opts.TitleOpts(
        title=f"{attr} Status for {len(states)} States"
      ),
      tooltip_opts=opts.TooltipOpts(trigger="axis"),
      yaxis_opts=opts.AxisOpts(
        type_="value",
        axistick_opts=opts.AxisTickOpts(is_show=True),
        splitline_opts=opts.SplitLineOpts(is_show=True)
      ),
      datazoom_opts=opts.DataZoomOpts(
        range_start=0,
        range_end=100
      ),
      legend_opts=opts.LegendOpts(is_show=True, pos_top=40)
    )
  )

  for (i, values) in enumerate(values_by_state):
    line.add_yaxis(
      series_name = state_dict[states[i]]["abbrev"].upper(),
      y_axis=values,
      label_opts=opts.LabelOpts(is_show=False),
    )

  grid = (
    Grid(init_opts=opts.InitOpts(width="775px", height="500px"))
      .add(line, grid_opts=opts.GridOpts(pos_top=175))
  ) 

  if save_name is not None:
    grid.render(os.path.join("./state/", save_name + ".html"))

  return grid


In [157]:
compare_states(
  attr="Age 0 to 4", 
  states=list(state_dict.keys()),
  save_name="age-0-4"
).render_notebook()